In [ ]:
"""
Add the zebrafish jaw lib to python path
"""

import os, sys

sys.path.append(os.path.abspath(os.path.join(os.path.dirname(""), "..", "..")))

In [ ]:
debug_plots = False

In [ ]:
"""
The first thing we'll do is read a model from disk - we want to access its config file

"""

from fishjaw.model import model

model_name = "attempt_3.pkl"
jaw_model = model.load_model("attempt_3.pkl")
jaw_config = jaw_model.config

In [ ]:
"""
We'll fine tune on some quadrates that Wahab segmented - we'll want to read these from the RDSF and crop to the right region of interest

"""

import re
import pathlib

import tifffile
from tqdm import tqdm

wahab_labels = (
    jaw_config["rdsf_dir"]
    / pathlib.Path("1Felix and Rich make models/Training dataset Tiffs/Training set 1")
).glob("*.tif")

# Remove these ones, since the 3D tifs dont exist
bad_labels = re.compile(r"(351|401|420|441)")
wahab_labels = [label for label in wahab_labels if not bad_labels.search(label.name)]

# Read the labels
quadrate_labels = [tifffile.imread(path) for path in tqdm(wahab_labels)]
quadrate_labels = [(l == 4) | (l == 5) for l in tqdm(quadrate_labels)]

In [ ]:
"""
Read in the images
"""

from fishjaw.util import files

img_paths = [files.get_3d_tif(label_path) for label_path in wahab_labels]
for p in img_paths:
    assert p.exists()

quadrate_imgs = [tifffile.imread(path) for path in tqdm(img_paths)]

In [ ]:
"""
Find the centre of the quadrates and crop the labels and images

"""

from scipy.ndimage import center_of_mass

centroids = [
    tuple(round(x) for x in center_of_mass(label)) for label in tqdm(quadrate_labels)  # type: ignore
]

In [ ]:
from fishjaw.images import transform

window_size = transform.window_size(jaw_config)
cropped_labels = [
    transform.crop(l, c, window_size, centred=True)
    for l, c in zip(tqdm(quadrate_labels), centroids)
]
cropped_quadrates = [
    transform.crop(i, c, window_size, centred=True)
    for i, c in zip(tqdm(quadrate_imgs), centroids)
]

In [ ]:
"""
Plot the jaws and labels just to check

"""

from fishjaw.visualisation import images_3d

if debug_plots:
    for img, label in zip(cropped_quadrates, cropped_labels):
        images_3d.plot_slices(img, label)

In [ ]:
"""
Create a dataloader for these

"""

import torchio as tio
from fishjaw.model import data


# This is the size of the training data
jaw_config["batch_size"] = 10

# Because we're in Jupyter
jaw_config["num_workers"] = 0

# Turn them all into tio subjects first
subjects = [
    data.imgs2subject(img, label)
    for img, label in zip(cropped_quadrates, cropped_labels)
]

train_subjects = tio.SubjectsDataset(
    subjects[:10], transform=data._transforms(jaw_config["transforms"])
)
val_subjects = tio.SubjectsDataset(
    [subjects[-2]], transform=data._transforms(jaw_config["transforms"])
)
test_subject = subjects[-1]

quadrate_data = data.DataConfig(jaw_config, train_subjects, val_subjects)

In [ ]:
"""
Plot the first bit of trainin data just to visualise it

"""

if debug_plots:
    for i, batch in enumerate(quadrate_data.train_data):
        images = batch[tio.IMAGE][tio.DATA]
        masks = batch[tio.LABEL][tio.DATA]
        # Images per batch
        for j, (image, mask) in enumerate(zip(images, masks)):
            fig, _ = images_3d.plot_slices(
                image.squeeze().numpy(), mask.squeeze().numpy()
            )

In [ ]:
"""
Plot the ground truth for the test data
"""

_ = images_3d.plot_subject(test_subject)

In [ ]:
"""
Train a model from scratch
"""

import torch


def train_new_model(
    data_config: data.DataConfig,
) -> tuple[
    tuple[torch.nn.Module, list[list[float]], list[list[float]]], torch.optim.Optimizer
]:
    """
    Create a new model, train and return it

    Returns the model, the training losses and the validation losses, and the optimiser

    """
    # Create a model and optimiser
    net = model.model(jaw_config["model_params"])
    net = net.to(jaw_config["device"])
    print(f"Model loaded to {jaw_config['device']}")

    optimiser = model.optimiser(jaw_config, net)

    # Define loss function
    loss = model.lossfn(jaw_config)

    train_config = model.TrainingConfig(
        jaw_config["device"],
        jaw_config["epochs"],
        torch.optim.lr_scheduler.ExponentialLR(
            optimiser, gamma=jaw_config["lr_lambda"]
        ),
    )
    return (
        model.train(net, optimiser, loss, data_config, train_config),
        optimiser,
    )

In [ ]:
jaw_config["epochs"] = 650
(net, train_losses, val_losses), optimiser = train_new_model(quadrate_data)

In [ ]:
torch.save(
    {
        "model_state_dict": net.state_dict(),
        "optimizer_state_dict": optimiser.state_dict(),
        "train_losses": train_losses,
        "val_losses": val_losses,
    },
    pathlib.Path(jaw_config["model_path"].replace(".pkl", "_quadrates.pth")),
)

In [ ]:
"""
Plot the training and validation losses

"""

import matplotlib.pyplot as plt
from fishjaw.visualisation import training

fig = training.plot_losses(train_losses, val_losses)
fig.savefig("new_model_losses.png")
plt.close(fig)

In [ ]:
"""
Plot the test subject
"""

fig = images_3d.plot_inference(
    net,
    test_subject,
    patch_size=data.get_patch_size(jaw_config),
    patch_overlap=(4, 4, 4),
    activation=model.activation_name(jaw_config),
    batch_size=jaw_config["batch_size"],
)
fig.savefig("new_model_inference.png")
plt.close(fig)

In [ ]:
"""
Now fine-tune the trained model on the testing data. it should perform better
"""

from monai.networks.nets.attentionunet import AttentionLayer


def fine_tune_model(
    data_config: data.DataConfig,
    train_layers: str = list[int],
    lr_multiplier: float = 0.1,  # Lower learning rate for fine-tuning
    epochs_frozen: int = 150,  # Train with frozen layers
    epochs_unfrozen: int = 50,  # Additional training with all layers
    verbose: bool = True,
) -> tuple[torch.nn.Module, list[list[float]], list[list[float]]]:
    """
    Fine-tune a model on the provided data

    :param train_layers: a list of integers: the layers to train
    """
    for l in train_layers:
        assert isinstance(l, int), f"Expected int, got {type(l)}"
        assert 0 <= l <= 5

    # Load the model from disk fresh so that we don't overwrite anything in memory
    new_model = model.load_model(model_name)
    net = new_model.load_model(set_eval=False)
    net.to(jaw_config["device"])

    # Freeze all the parameters
    for param in net.parameters():
        param.requires_grad = False

    # Get the bits of the model
    head = net.model[0]  # ConvBlock
    encdec = net.model[1]  # The recursive AttentionLayer structure
    final_conv = net.model[2]  # Final convolution

    def unfreeze_attention_layers(
        module: torch.nn.Module, current_depth: int, target_depths: list[int]
    ):
        """
        Recursively unfreeze AttentionLayer parameters at specified depths
        """
        if isinstance(module, AttentionLayer):
            if current_depth in target_depths:
                if verbose:
                    print(f"Unfreezing AttentionLayer at depth {current_depth}")
                for name, param in module.named_parameters():
                    if "submodule" not in name:
                        param.requires_grad = True
                        if verbose:
                            print("  Unfreezing:", name)

            # Recurse into submodule
            if hasattr(module, "submodule"):
                unfreeze_attention_layers(
                    module.submodule, current_depth + 1, target_depths
                )

        # Handle Sequential containers (like in submodule)
        elif isinstance(module, torch.nn.Sequential):
            for child in module.children():
                unfreeze_attention_layers(child, current_depth, target_depths)

    unfreeze_attention_layers(encdec, current_depth=0, target_depths=train_layers)
    if verbose:
        trainable_params = 0
        total_params = 0
        for name, param in net.named_parameters():
            total_params += param.numel()
            if param.requires_grad:
                trainable_params += param.numel()
                print(f"Trainable: {name} ({param.numel()} params)")

        print(f"Total trainable parameters: {trainable_params:,} / {total_params:,}")

    # Create a new optimiser that only updates the unfrozen layers
    # Get the right optimiser from the config
    # and set the learning rate to a lower value
    optimiser = getattr(torch.optim, jaw_config["optimiser"])(
        (p for p in net.parameters() if p.requires_grad),
        lr=jaw_config["learning_rate"] * lr_multiplier,
    )

    # Create a loss function
    loss = model.lossfn(jaw_config)

    # Train the model with the frozen layers
    train_config = model.TrainingConfig(
        jaw_config["device"],
        epochs_frozen,
        torch.optim.lr_scheduler.ExponentialLR(
            optimiser, gamma=jaw_config["lr_lambda"]
        ),
    )

    print(f"Training with selective freezing for {epochs_frozen} epochs...")
    return model.train(net, optimiser, loss, data_config, train_config)

In [ ]:
"""
First demonstrate what the fine tuning looks like if we basically dont train it at all
"""

fine_tuned_model, fine_tune_train_losses, fine_tune_val_losses = fine_tune_model(
    quadrate_data,
    train_layers=[1, 2],
    lr_multiplier=2.0,
    epochs_frozen=2,
    epochs_unfrozen=1,
)

In [ ]:
_ = images_3d.plot_inference(
    fine_tuned_model,
    test_subject,
    patch_size=data.get_patch_size(jaw_config),
    patch_overlap=(4, 4, 4),
    activation=model.activation_name(jaw_config),
    batch_size=jaw_config["batch_size"],
)

In [ ]:
"""
Now train it more properly
"""

fine_tuned_model, fine_tune_train_losses, fine_tune_val_losses = fine_tune_model(
    quadrate_data,
    train_layers=[1, 2],
    lr_multiplier=2.0,
    epochs_frozen=100,
    epochs_unfrozen=50,
)

In [ ]:
from fishjaw.visualisation import training

training.plot_losses(fine_tune_train_losses, fine_tune_val_losses)
_ = images_3d.plot_inference(
    fine_tuned_model,
    test_subject,
    patch_size=data.get_patch_size(jaw_config),
    patch_overlap=(4, 4, 4),
    activation=model.activation_name(jaw_config),
    batch_size=jaw_config["batch_size"],
)

In [ ]:
"""
We might want to compare the weights of the original model and the fine-tuned model, to see where the changes are
"""


def get_weight_deltas(
    model_before, model_after
) -> tuple[dict[str, torch.Tensor], dict[str, torch.Tensor]]:
    """
    Get the difference in weights between two models, and the original weights
    """
    deltas = {}
    orig_weights = {}
    for (name1, param1), (name2, param2) in zip(
        model_before.named_parameters(), model_after.named_parameters()
    ):
        assert name1 == name2, f"Names do not match: {name1} != {name2}"

        if param1.requires_grad and "num_batches_tracked" not in name1:
            delta = param2.data - param1.data
            deltas[name1] = delta

            orig_weights[name1] = param1.data
    return deltas, orig_weights

In [ ]:
"""
Plot histograms of all of the weight deltas
"""

deltas, orig_weights = get_weight_deltas(
    jaw_model.load_model().to(jaw_config["device"]), fine_tuned_model
)

In [ ]:
import textwrap
import numpy as np
import matplotlib.pyplot as plt

# Only do this sometimes because it's a really big image
if debug_plots:
    bins = np.linspace(
        torch.min(torch.cat(list(d.flatten() for d in deltas.values()))).item(),
        torch.max(torch.cat(list(d.flatten() for d in deltas.values()))).item(),
        100,
    )

    fig, axes = plt.subplots(
        len(deltas) // 4, 4, figsize=(8, len(deltas) // 4 * 2), sharey=True
    )
    for axis, (name, delta) in zip(axes.flatten(), tqdm(deltas.items())):
        axis.hist(delta.flatten().cpu().numpy(), bins=bins, density=True)
        axis.set_title("\n".join(textwrap.wrap(name, 30)), fontsize=8)
        # axis.set_yscale("log")

    fig.tight_layout()

In [ ]:
"""
We want a way to isolate each type of weight (conv, psi, merge, etc)
"""

import re

weight_type_regex = {
    "down_conv_0_weight": r".*conv.0.conv.weight",
    "down_conv_0_bias": r".*conv.0.conv.bias",
    "down_conv_1_weight": r".*conv.1.conv.weight",
    "down_conv_1_bias": r".*conv.1.conv.bias",
    "down_adn_0_weight": r".*conv.0.adn.N.weight",
    "down_adn_0_bias": r".*conv.0.adn.N.bias",
    "down_adn_1_weight": r".*conv.1.adn.N.weight",
    "down_adn_1_bias": r".*conv.1.adn.N.bias",
    "attention_wg_0_weight": r".*attention.W_g.0.conv.weight",
    "attention_wg_0_bias": r".*attention.W_g.0.conv.bias",
    "attention_wg_1_weight": r".*attention.W_g.1.weight",
    "attention_wg_1_bias": r".*attention.W_g.1.bias",
    "attention_wx_0_weight": r".*attention.W_x.0.conv.weight",
    "attention_wx_0_bias": r".*attention.W_x.0.conv.bias",
    "attention_wx_1_weight": r".*attention.W_x.1.weight",
    "attention_wx_1_bias": r".*attention.W_x.1.bias",
    "attention_psi_0_weight": r".*attention.psi.0.conv.weight",
    "attention_psi_0_bias": r".*attention.psi.0.conv.bias",
    "attention_psi_1_weight": r".*attention.psi.1.weight",
    "attention_psi_1_bias": r".*attention.psi.1.bias",
    "upconv_weight": r".*upconv.up.conv.weight",
    "upconv_bias": r".*upconv.up.conv.bias",
    "upconv_adn_weight": r".*upconv.up.adn.N.weight",
    "upconv_adn_bias": r".*upconv.up.adn.N.bias",
    "merge_weight": r".*merge.conv.weight",
    "merge_bias": r".*merge.conv.bias",
    "merge_adn": r".*merge.adn.A.weight",
}


if debug_plots:
    for name, regex in weight_type_regex.items():
        total = 0
        for k in deltas.keys():
            if re.search(regex, k):
                total += 1
        print(f"{name}: {total} matches")

In [ ]:
"""
Make a u-net style diagram, where each histogram is in a different part of the u-net, and the skip connections are drawn as arrows

As much as I love this, it's ridiculous

"""

from matplotlib.patches import FancyArrowPatch


def _draw_skip_arrows(fig: plt.Figure, axes: dict) -> None:
    """
    draw arrows
    """
    # Define the skip connections (encoder to bottleneck to decoder)
    skip_connections = [
        ("A", "a", "K"),  # Encoder level 1 → Bottleneck → Decoder level 1
        ("B", "b", "J"),  # Encoder level 2 → Bottleneck → Decoder level 2
        ("C", "c", "I"),  # Encoder level 3 → Bottleneck → Decoder level 3
        ("D", "d", "H"),  # Encoder level 4 → Bottleneck → Decoder level 4
        ("E", "e", "G"),  # Encoder level 5 → Bottleneck → Decoder level 5
    ]

    # Add arrows for skip connections
    arrow_params = dict(
        connectionstyle="arc3,rad=-0.3",
        arrowstyle="simple,head_length=5,head_width=5",
        linewidth=0.5,
        transform=fig.transFigure,
        color="k",
    )
    for encoder, bottleneck, decoder in skip_connections:
        # Get positions of axes
        encoder_pos = axes[encoder].get_position()
        attn_pos = axes[bottleneck].get_position()
        decoder_pos = axes[decoder].get_position()

        # Calculate arrow coordinates
        # Encoder to bottleneck arrow
        x1 = encoder_pos.x1  # Right side of encoder
        y1 = encoder_pos.y0 + 0.75 * encoder_pos.height  # 3/4 up
        x2 = attn_pos.x0  # Left side of bottleneck
        y2 = attn_pos.y0 + 0.5 * attn_pos.height  # Middle

        # Draw arrow
        fig.patches.extend([FancyArrowPatch((x1, y1), (x2, y2), **arrow_params)])

        # Bottleneck to decoder arrow
        x1 = attn_pos.x1  # Right side of bottleneck
        y1 = attn_pos.y0 + 0.5 * attn_pos.height  # Middle
        x2 = decoder_pos.x0  # Left side of decoder
        y2 = decoder_pos.y0 + 0.75 * decoder_pos.height  # 3/4 up

        fig.patches.extend([FancyArrowPatch((x1, y1), (x2, y2), **arrow_params)])


def unet_hists(weight_type: str) -> plt.Figure:
    """
    Draw histograms in a u-net shape for weights matching the given pattern
    """

    # Get the layer names for the given weight type
    pattern = weight_type_regex[weight_type]
    names = []
    for k in deltas.keys():
        if re.search(pattern, k):
            names.append(k)

    fig, axes = plt.subplot_mosaic(
        """
        AA.........aKK
        AA..........KK
        .BB.......bJJ.
        .BB........JJ.
        ..CC.....cII..
        ..CC......II..
        ...DD...dHH...
        ...DD....HH...
        ....EE.eGG....
        ....EE..GG....
        ......FF......
        ......FF......
        """,
        figsize=(15, 15),
    )

    _draw_skip_arrows(fig, axes)

    # Choose the axes to plot on
    down_axes = "ABCDEF"
    up_axes = "GHIJK"
    attn_axes = "abcde"

    if "upconv" in weight_type or "merge" in weight_type:
        plot_axes = {k: axes[k] for k in up_axes}
    elif "attention" in weight_type:
        plot_axes = {k: axes[k] for k in attn_axes}
    else:
        assert "down" in weight_type
        plot_axes = {k: axes[k] for k in down_axes}

    deltas_ = [deltas[k] for k in names]
    max_ = max([d.abs().max().cpu() for d in deltas_])
    bins = np.linspace(-max_, max_, 100)
    for name, delta, axis in zip(names, deltas_, plot_axes.values()):
        # Get the delta
        delta = delta.flatten().cpu().numpy()

        # Plot the histogram
        axis.hist(delta, bins=bins, density=True)
        axis.set_yscale("log")

        axis.axvline(
            0,
            color="k",
            linestyle="--",
            linewidth=0.5,
        )

    # Remove axis ticks on the unused axes
    for axis in axes.values():
        if axis not in plot_axes.values():
            axis.set_xticks([])
            axis.set_yticks([])

    fig.suptitle(f"$\Delta${weight_type}")
    return fig

In [ ]:
"""
Boxplots are a more sensible way to visualise things, I think
"""


def boxplots(deltas: dict) -> plt.Figure:
    """
    Make a boxplot with the weight deltas for a given weight type
    """

    # There are 27 weight types, I think
    fig, axes = plt.subplots(9, 3, figsize=(9, 27), sharex=True, sharey=True)

    for axis, weight_type in zip(
        tqdm(axes.flatten()), weight_type_regex.keys(), strict=True
    ):
        pattern = weight_type_regex[weight_type]
        names = []
        for k in deltas.keys():
            if re.search(pattern, k):
                names.append(k)

        for i, d in enumerate([deltas[k] for k in names]):
            axis.boxplot(
                d.flatten().cpu().numpy(),
                positions=[i],
                widths=0.5,
                vert=True,
                patch_artist=True,
            )
            axis.set_title(weight_type)
            axis.axhline(
                0,
                color="k",
                linestyle="--",
                linewidth=0.5,
            )

    fig.suptitle("$\Delta$ weight")
    fig.supxlabel("Layer")
    fig.supylabel("Change in weight")

    fig.tight_layout()
    return fig

In [ ]:
fig = boxplots(deltas)
plt.show()
plt.close(fig)

In [ ]:
"""
Train all the layers
"""

fine_tuned_model, fine_tune_train_losses, fine_tune_val_losses = fine_tune_model(
    quadrate_data,
    train_layers=[0, 1, 2, 3, 4],
    lr_multiplier=0.5,
    epochs_frozen=200,
    epochs_unfrozen=50,
    verbose=False,
)

training.plot_losses(fine_tune_train_losses, fine_tune_val_losses)
_ = images_3d.plot_inference(
    fine_tuned_model,
    test_subject,
    patch_size=data.get_patch_size(jaw_config),
    patch_overlap=(4, 4, 4),
    activation=model.activation_name(jaw_config),
    batch_size=jaw_config["batch_size"],
)

fig = boxplots(
    get_weight_deltas(
        jaw_model.load_model().to(jaw_config["device"]), fine_tuned_model
    )[0]
)
plt.show()
plt.close(fig)

In [ ]:
"""
Do the above with every two-layer combination, to see if we can get a better result
"""

from itertools import combinations
from fishjaw.util import files

out_dir = files.script_out_dir() / "fine_tune_quadrates"
if not out_dir.exists():
    out_dir = pathlib.Path("script_output")
    out_dir.mkdir(parents=True)

layers = [0, 1, 2, 3, 4]
all_combinations = [
    list(combo) for i in range(1, len(layers) + 1) for combo in combinations(layers, i)
]

for train_layers in tqdm(all_combinations):
    print(f"Training layers: {train_layers}")
    this_dir = out_dir / "_".join(map(str, train_layers))
    if not this_dir.exists():
        this_dir.mkdir(parents=True)

    fine_tuned_model, fine_tune_train_losses, fine_tune_val_losses = fine_tune_model(
        quadrate_data,
        train_layers=train_layers,
        lr_multiplier=0.5,
        epochs_frozen=200,
        epochs_unfrozen=50,
        verbose=False,
    )
    fig = training.plot_losses(fine_tune_train_losses, fine_tune_val_losses)
    fig.savefig(this_dir / f"losses.png")
    plt.close(fig)

    fig = images_3d.plot_inference(
        fine_tuned_model,
        test_subject,
        patch_size=data.get_patch_size(jaw_config),
        patch_overlap=(4, 4, 4),
        activation=model.activation_name(jaw_config),
        batch_size=jaw_config["batch_size"],
    )
    fig.savefig(this_dir / f"inference.png")
    plt.close(fig)

    fig = boxplots(
        get_weight_deltas(
            jaw_model.load_model().to(jaw_config["device"]), fine_tuned_model
        )[0]
    )
    fig.savefig(this_dir / f"weight_deltas.png")
    plt.close(fig)